In [2]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from dotenv import get_key
import os
from math import ceil

In [3]:
# file_path = (
#     "./files/first_chapter.pdf"
# )
file_path = (
    "./files/2024-dbir-data-breach-investigations-report.pdf"
)
file_name = '2024-dbir-data-breach-investigations-report.pdf'
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()

In [4]:
os.environ['GOOGLE_API_KEY'] = get_key('.env', 'GEMINI_API_KEY')

In [5]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators=['.'])
docs = text_splitter.split_documents(pages)

In [63]:
db_local = None
 
for i in range(0, len(docs), 100):
    if i + 100 < len(docs):
        db = FAISS.from_documents(docs[i:i+100], embeddings)
    else: 
        db = FAISS.from_documents(docs[i:len(docs)], embeddings)
    if not db_local:
        db_local = db
    else: 
        db_local.merge_from(db) 
db_local.save_local(f'./faiss_db/{file_name}')

In [66]:
db = FAISS.load_local(f"./faiss_db/{file_name}", embeddings, allow_dangerous_deserialization=True)

In [67]:
db.index.ntotal

287

In [73]:
summarization_percentage = 0.5

In [74]:
k = summarization_percentage * db.index.ntotal
k = ceil(k)
k

144

In [80]:
# retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
retriever = db.as_retriever(search_type="similarity")

In [86]:
retrieved_docs = retriever.invoke("secure")

In [87]:
text = []
for doc in retrieved_docs:
    text.append(doc.page_content)

text
len(text)

4

In [ ]:
llm = genai.GenerativeModel(
    model_name='gemini-pro'
)